# Создание базы данных

In [85]:
import sqlite3

In [21]:
create_queries = ['''
CREATE TABLE "texts" ( 
	"id"	INTEGER,
	"name"	TEXT UNIQUE,
	PRIMARY KEY("id" AUTOINCREMENT)
);''', 
'''
CREATE TABLE "sents" (
	"id"	INTEGER,
	"text_id"	INTEGER,
	"sent"	TEXT,
	"lemmatized"	TEXT,
	"pos_in_text"	INTEGER,
	FOREIGN KEY("text_id") REFERENCES "texts"("id"),
	PRIMARY KEY("id" AUTOINCREMENT)
);''', 

'''
CREATE TABLE "govern models" (
	"id"	INTEGER,
	"model"	TEXT,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "math_imgs" (
	"id"	INTEGER,
	"math_tagname"	TEXT,
	"filename"	TEXT,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "math_tags" (
	"id"	INTEGER,
	"feature"	TEXT,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "lemmas" (
	"id"	INTEGER,
	"name"	TEXT UNIQUE,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "pos" (
	"id"	INTEGER,
	"name"	TEXT,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "deprels" (
	"id"	INTEGER,
	"name"	INTEGER,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "tokens" (
	"id"	INTEGER,
	"sent_id"	INTEGER,
	"word_in_sent"	INTEGER,
	"token"	TEXT,
	FOREIGN KEY("sent_id") REFERENCES "sents"("id") ON DELETE RESTRICT,
	PRIMARY KEY("id" AUTOINCREMENT)
);
''', 

'''
CREATE TABLE "math_annotation" (
	"id"	INTEGER,
	"id_token_start"	INTEGER,
	"id_token_end"	INTEGER,
	"feature_id"	INTEGER,
	"govern_model_id"	INTEGER,
	FOREIGN KEY("feature_id") REFERENCES "math_tags"("id"),
	FOREIGN KEY("govern_model_id") REFERENCES "govern models"("id"),
	FOREIGN KEY("id_token_start") REFERENCES "tokens"("id"),
	FOREIGN KEY("id_token_end") REFERENCES "tokens"("id"),
	PRIMARY KEY("id" AUTOINCREMENT)
);
''',

'''
CREATE TABLE "grammar_annotation" (
	"token_id"	INTEGER,
	"lemma_id"	INTEGER,
	"pos_id"	INTEGER,
	"deprel_id"	INTEGER,
	"head_id"	INTEGER,
	FOREIGN KEY("lemma_id") REFERENCES "lemmas"("id"),
	FOREIGN KEY("token_id") REFERENCES "tokens"("id"),
	FOREIGN KEY("deprel_id") REFERENCES "deprels"("id"),
	FOREIGN KEY("head_id") REFERENCES "tokens"("id"),
	FOREIGN KEY("pos_id") REFERENCES "pos"("id"),
	PRIMARY KEY("token_id")
);
'''
                 ]

In [23]:
# db_path = 'math_corpus_database.db'
# conn = sqlite3.connect(db_path, check_same_thread=False)
# cur = conn.cursor()

In [103]:
# for q in create_queries:
#     cur.execute(q)
#     conn.commit()
conn.close()

In [86]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 15.3/15.3 MB 11.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [89]:
db_path = 'math_corpus_database.db'

In [90]:
class DBHandler:
    conn = None
    cur = None

    def __init__(self, db_path):
        print('init')
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cur = self.conn.cursor()

    def __del__(self):
        self.conn.close()

    def add_text(self, text_name):
        
        self.cur.execute('''INSERT or IGNORE INTO texts (name)
                            VALUES (?)
                            RETURNING id''', (text_name,))
    
        added_id = self.cur.fetchone()
        self.conn.commit()
        
        if not added_id:
            self.cur.execute('''SELECT id
                                FROM texts
                                WHERE name = (?)''', (text_name, ))
            added_id = self.cur.fetchone()
        return added_id[0]
        
    def add_sent(self, text_id, sent, lemmatized, sent_count):
        
        self.cur.execute('''INSERT INTO sents (text_id, sent, lemmatized, pos_in_text)
                            VALUES (?, ?, ?, ?)
                            RETURNING id''', (text_id, sent, lemmatized, sent_count))
        
        added_id = self.cur.fetchone()
        self.conn.commit()
        return added_id[0]
    
    def add_tokens(self, tokens_info): 
        ":param tokens_info: list of tuples (sent_id, word_in_sent, token)"
        
        self.cur.executemany('''INSERT INTO tokens (sent_id, word_in_sent, token)
                                VALUES (?, ?, ?)''', tokens_info)
        
        self.conn.commit()
        
        self.cur.execute('''SELECT id 
                            FROM tokens
                            WHERE sent_id = (?)
                            ORDER BY word_in_sent''', (tokens_info[0][0],))
        added_ids = self.cur.fetchall()
        return [token_id[0] for token_id in added_ids]
    
    
    def add_lemmas(self, lemmas):
        lemmas = [(el,) for el in lemmas]
        self.cur.executemany('''INSERT or IGNORE
                            INTO lemmas (name)
                            VALUES (?)''', lemmas)
        self.conn.commit()
        
        lemma_ids = []
        for lemma in lemmas:
            print(lemma)
            self.cur.execute('''SELECT id
                                FROM lemmas
                                WHERE name = ?''', lemma)
            lemma_ids.append(self.cur.fetchone()[0])
        return lemma_ids
    
    
    def add_grammar_annot(self, token_grammar_info):
        ":param token_grammar_info: [(token_id, pos, deprel, head_id, lemma)]"
        
        self.cur.executemany('''INSERT INTO grammar_annotation (token_id, lemma_id, pos_id, deprel_id, head_id)
                                SELECT ?, 
                                        lemmas.id, 
                                        (SELECT pos.id FROM pos WHERE pos.name = ?),
                                        (SELECT deprels.id FROM deprels WHERE deprels.name = ?),
                                        ?
                                FROM lemmas
                                WHERE lemmas.name = ?''', token_grammar_info)
        self.conn.commit()
db = DBHandler(db_path)

init


In [91]:
xml_filename = 'test.xml'

In [92]:
from bs4 import BeautifulSoup

In [93]:
with open(xml_filename, 'r', encoding='utf-8') as f:
    xml_file = f.read()
bs_data = BeautifulSoup(xml_file, "xml")
segs = bs_data.find_all('segment')

In [115]:
def parse_sentence(analysed_sent):
    parsed = {
        'tokens': [], 
        'lemmas': [], 
        'poses': [], 
        'deprels': [], 
        'head_id_in_sent': []
    }
    first_word_in_sent = 0
    
    for t in analysed_sent:
        parsed['tokens'].append(t.text)
        parsed['lemmas'].append(t.lemma_)
        parsed['poses'].append(t.pos_)
        parsed['deprels'].append(t.dep_.lower())
        
        if t.is_sent_start:
            first_word_in_sent = t.i
        
        parsed['head_id_in_sent'].append(t.head.i - first_word_in_sent)
        
    return parsed

In [116]:
def accum_token_info(parsed, sent_id):
    n_tokens = len(parsed['tokens'])
    return [(sent_id, i, parsed['tokens'][i]) for i in range(n_tokens)]

In [118]:
def accum_grammar_info(parsed, added_tokens):
    n_tokens = len(parsed['tokens'])
#     print('added_tokens: ', added_tokens, len(added_tokens))
#     print('heads: ', parsed['head_id_in_sent'], len(parsed['head_id_in_sent']))
    head_ids = [added_tokens[hi] for hi in parsed['head_id_in_sent']]
    token_grammar_info = [
    (
        added_tokens[i], 
        parsed['poses'][i],
        parsed['deprels'][i], 
        head_ids[i],
        parsed['lemmas'][i]
    ) for i in range(n_tokens)
    ]
    return token_grammar_info

In [119]:
text_id = 1
sent_count = 0
for seg in segs:
    parent_id = seg.get('parent')
    if not parent_id:
        text = seg.get_text()
        sentences = list(nlp(text).sents)
        for sent in sentences:
            sent_count += 1
            parsed_sentence = parse_sentence(sent)
            
            lemmas = [parsed_sentence['lemmas'][i] for i in range(len(parsed_sentence['tokens']))]
            added_lemmas = db.add_lemmas(lemmas)
            
            sent_text = sent.text
            sent_lemmatized = ' '.join([lemma for lemma in lemmas])
            sent_id = db.add_sent(text_id, sent_text, sent_lemmatized, sent_count)
            
            tokens_info = accum_token_info(parsed_sentence, sent_id)
            added_tokens = db.add_tokens(tokens_info)
    
            
            grammar_info = accum_grammar_info(parsed_sentence, added_tokens)
            db.add_grammar_annot(grammar_info)
    else:
        print()
        print(added_lemmas)
        print(added_tokens)
        print(seg.get_text(), seg.get('features'))

('степень',)
('с',)
('натуральный',)
('показатель',)
('.',)

[241, 242, 243, 244, 245]
[184, 185, 186, 187, 188]
Степень math_tags;oper;power

[241, 242, 243, 244, 245]
[184, 185, 186, 187, 188]
натуральным math_tags;term;set;nat

[241, 242, 243, 244, 245]
[184, 185, 186, 187, 188]
показателем math_tags;visual;superscript
('определение',)
('степень',)
(':',)
('а',)
('в',)
('энной',)
('степень',)
('равняться',)
('произведение',)
('энн',)
('множитель',)
(',',)
('каждый',)
('из',)
('которых',)
('равный',)
('а',)
(',',)
('где',)
('энн',)
('-',)
('число',)
('натуральный',)
('.',)
('считаться',)
(',',)
('что',)
('а',)
('в',)
('первый',)
('степень',)
('равняться',)
('а',)
('.',)

[270, 257, 272, 249, 250, 275, 241, 253, 249, 245]
[213, 214, 215, 216, 217, 218, 219, 220, 221, 222]
Определение math_tags;comment;definition

[270, 257, 272, 249, 250, 275, 241, 253, 249, 245]
[213, 214, 215, 216, 217, 218, 219, 220, 221, 222]
степени math_tags;oper;power

[270, 257, 272, 249, 250, 275, 241, 253, 2


[329, 330, 331, 332, 333, 257, 280, 248, 337, 250, 251, 241, 261, 342, 257, 344, 250, 251, 241, 261, 344, 305, 351, 243, 267, 354, 257, 337, 250, 358, 241, 266, 361, 362, 257, 293, 365, 361, 362, 245]
[272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311]
ноль math_tags;term;number;common;self

[329, 330, 331, 332, 333, 257, 280, 248, 337, 250, 251, 241, 261, 342, 257, 344, 250, 251, 241, 261, 344, 305, 351, 243, 267, 354, 257, 337, 250, 358, 241, 266, 361, 362, 257, 293, 365, 361, 362, 245]
[272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311]
энной math_tags;term;var;common;letter;latin;n

[329, 330, 331, 332, 333, 257, 280, 248, 337, 250, 251, 241, 261, 342, 257, 344, 250, 251, 241, 261, 344, 3

('в',)
('наш',)
('пример',)
('основание',)
('степень',)
('одинаковый',)
(',',)
('число',)
('двойка',)
(',',)
('для',)
('того',)
(',',)
('чтобы',)
('их',)
('перемножить',)
(',',)
('такой',)
('степень',)
(',',)
('необходимый',)
('показатель',)
('сложить',)
(':',)
('пять',)
('плюс',)
('три',)
('равный',)
('восемь',)
(',',)
('получить',)
('два',)
('в',)
('восьмой',)
('степень',)
('.',)
('второй',)
('свойство',)
(':',)
('при',)
('деление',)
('степень',)
('с',)
('одинаковый',)
('основание',)
('показатель',)
('вычитать',)
('.',)

[302, 333, 248, 417, 470, 241, 242, 421, 422, 244, 476, 245]
[409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420]
два math_tags;term;number;common;self

[302, 333, 248, 417, 470, 241, 242, 421, 422, 244, 476, 245]
[409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420]
пятой math_tags;term;number;common;self

[302, 333, 248, 417, 470, 241, 242, 421, 422, 244, 476, 245]
[409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420]
степени math_tags;oper;po

In [13]:
segs

[<segment features="math_tags" filename="" id="1" state="active" subcorp=""><segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment> с <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment> <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>.</segment>,
 <segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment>,
 <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment>,
 <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>,
 <segment feat